In [ ]:
# !pip install "sap-llm-commons[all]"

In [ ]:
# !pip install ai-api-client-sdk

In [9]:
# proxy configuration
# from ipywidgets import widgets
import json
import os
import requests
import llm_commons.proxy.base

llm_commons.proxy.base.proxy_version = 'aicore'

In [10]:
from llm_commons.proxy.openai import Completion
from llm_commons.proxy.identity import AICoreProxyClient
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.btp_llm.identity import BTPProxyClient
from llm_commons.langchain.proxy import init_llm, init_embedding_model

In [ ]:
# resource_group = widgets.Text(
#     value='default', # resource group
#     placeholder='Resource group of deployments',
#     description='',
#     disabled=False
# )

# resource_group

In [11]:
with open('irpa-d26-genaixl-cx-sec-cn-sk.json') as f:
    sk = json.load(f)

In [13]:
os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
# os.environ['AICORE_LLM_RESOURCE_GROUP'] = resource_group.value
os.environ['LLM_COMMONS_PROXY'] = 'aicore'
os.environ['ENDPOINT_URL'] = sk['endpoint']

llm_commons.proxy.resource_group = 'default'
llm_commons.proxy.api_base = os.environ['AICORE_LLM_API_BASE']
llm_commons.proxy.auth_url = os.environ['AICORE_LLM_AUTH_URL']
llm_commons.proxy.client_id = os.environ['AICORE_LLM_CLIENT_ID']
llm_commons.proxy.client_secret = os.environ['AICORE_LLM_CLIENT_SECRET']
llm_commons.proxy.endpoint_url = os.environ['ENDPOINT_URL']

In [14]:
aic_proxy_client = AICoreProxyClient()


In [15]:
response = requests.post(
        f'{os.environ["AICORE_LLM_AUTH_URL"]}/oauth/token',
        data={"grant_type": "client_credentials"},
        auth=(os.environ['AICORE_LLM_CLIENT_ID'], os.environ['AICORE_LLM_CLIENT_SECRET']),
        timeout=8000,
)
auth_token = response.json()["access_token"]
print (auth_token)

eyJhbGciOiJSUzI1NiIsImprdSI6Imh0dHBzOi8vc2FwaXQtY29yZS1wbGF5Z3JvdW5kLXZvbGUuYXV0aGVudGljYXRpb24uZXUxMC5oYW5hLm9uZGVtYW5kLmNvbS90b2tlbl9rZXlzIiwia2lkIjoiZGVmYXVsdC1qd3Qta2V5LTEyNTI4Njk5MzEiLCJ0eXAiOiJKV1QiLCJqaWQiOiAiOHE0QTBxWjBYeDdFODRubmhxbVN5SlpyTFRteEo0MC9KMGs0c0JCTUIrYz0ifQ.eyJqdGkiOiJlNWJjZjEwZDZhNjQ0NmUzOTc2Y2YzMDJhYWU3NDczNSIsImV4dF9hdHRyIjp7ImVuaGFuY2VyIjoiWFNVQUEiLCJzdWJhY2NvdW50aWQiOiIyYWFlZmQ4Mi1iMWZlLTRiNTYtYWEyMi04NTFjMGUyNzE2MDQiLCJ6ZG4iOiJzYXBpdC1jb3JlLXBsYXlncm91bmQtdm9sZSIsInNlcnZpY2VpbnN0YW5jZWlkIjoiNzk3MGIxMjktM2ZlZi00MTAyLTk0NzItYjQ5YTlkZWQ0MGY5In0sInN1YiI6InNiLTc5NzBiMTI5LTNmZWYtNDEwMi05NDcyLWI0OWE5ZGVkNDBmOSFiMzEzMDkxfGFpc3ZjLTY2MjMxOGY5LWllcy1haWNvcmUtc2VydmljZSFiNTQwIiwiYXV0aG9yaXRpZXMiOlsiYWlzdmMtNjYyMzE4ZjktaWVzLWFpY29yZS1zZXJ2aWNlIWI1NDAub2JqZWN0c3RvcmVzZWNyZXQuY3JlZGVudGlhbHMuZGVsZXRlIiwiYWlzdmMtNjYyMzE4ZjktaWVzLWFpY29yZS1zZXJ2aWNlIWI1NDAuc2NlbmFyaW9zLmNvbmZpZ3VyYXRpb25zLnJlYWQiLCJhaXN2Yy02NjIzMThmOS1pZXMtYWljb3JlLXNlcnZpY2UhYjU0MC5vYmplY3RzdG9yZXNlY3JldC5jc

Deploy customized fine-tuned models

In [16]:
import autogen

llm_config={
    "config_list":[
        {
            # "base_url":"is the same with restful api url api_key is the restful api auth token"
            "base_url":llm_commons.proxy.endpoint_url,
            "api_key": auth_token,
            "api_version":"2023-05-15",
            "api_type":"azure"
        }
    ],
    "temperature": 0,
    "model":"gpt-4"
}

In [17]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
user_proxy= autogen.UserProxyAgent(
    name="user_proxy",
    max_consecutive_auto_reply=10,
    human_input_mode="NEVER",
    llm_config=llm_config,
       code_execution_config={
        "use_docker": False,
    },
)

In [18]:
user_proxy.initiate_chat(
    assistant,
    message="""what is the whether in shanghai?""",
)

user_proxy (to assistant):

what is the whether in shanghai?

--------------------------------------------------------------------------------
assistant (to user_proxy):

To get the weather information, we can use the OpenWeatherMap API. It provides weather data for any location in the world. However, it requires an API key which you can get for free by creating an account on their website.

Here is a Python script that uses the requests library to send a GET request to the OpenWeatherMap API and print the weather information for Shanghai. Replace 'your_api_key' with your actual API key.

```python
import requests
import json

def get_weather(city_name, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city_name,
        'appid': api_key,
        'units': 'metric'
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    print(f"Weather in {city_name}:")
    print(f"Temperature: {data['main']['temp']}°C

ChatResult(chat_history=[{'content': 'what is the whether in shanghai?', 'role': 'assistant'}, {'content': 'To get the weather information, we can use the OpenWeatherMap API. It provides weather data for any location in the world. However, it requires an API key which you can get for free by creating an account on their website.\n\nHere is a Python script that uses the requests library to send a GET request to the OpenWeatherMap API and print the weather information for Shanghai. Replace \'your_api_key\' with your actual API key.\n\n```python\nimport requests\nimport json\n\ndef get_weather(city_name, api_key):\n    base_url = "http://api.openweathermap.org/data/2.5/weather"\n    params = {\n        \'q\': city_name,\n        \'appid\': api_key,\n        \'units\': \'metric\'\n    }\n    response = requests.get(base_url, params=params)\n    data = response.json()\n    print(f"Weather in {city_name}:")\n    print(f"Temperature: {data[\'main\'][\'temp\']}°C")\n    print(f"Weather: {data[